In [101]:
from datetime import date
import pandas as pd
import numpy as np
import copy
import sqlite3
conn = sqlite3.connect('data/database.sqlite')
pd.set_option('display.max_columns', 500)



# Create dataframe of relevant matches  with details about shot on target and goals. Only matches without red cards

In [102]:
# 10, 15, 20, 25
threshold_over_50 = 10
poss_name = 'possession_until_halftime'
Y_column = 'shots_on_second_half'
file_name = 'data/statistics_no_tie_matches_poss_first_half.csv'

In [103]:
df = pd.read_csv(file_name)
df = df[abs(df[poss_name] - 50) >= threshold_over_50]
# df = df.groupby("match_id").apply(pd.DataFrame.sample, n=1)

df['T'] = df[poss_name] >= (50+threshold_over_50)

df = df.drop(columns=["Unnamed: 0", 'shots_on', 'goals', 'goals_second_half', poss_name])
df = df.reset_index()
df.head()

,index,match_id,team_id,shots_on_first_half,shots_on_second_half,goals_first_half,T
0,2,1730,9825,7,5,1,True
1,3,1730,8659,0,2,0,False
2,16,1738,8462,5,2,3,False
3,17,1738,8455,5,1,0,True
4,48,1757,9825,3,1,1,True


In [104]:
df_matches = pd.read_sql("""SELECT * FROM Match;""", conn)
# df_matches = pd.read_csv("match.csv")
df_matches = df_matches.drop(columns=['PSH','PSD','PSA','foulcommit','shotoff','shoton','goal','card','cross','corner','possession','home_team_goal','away_team_goal','match_api_id','home_player_X1','home_player_X2','home_player_X3','home_player_X4','home_player_X5',
                                      'home_player_X6','home_player_X7','home_player_X8','home_player_X9','home_player_X10','home_player_X11',
                                     'away_player_X1','away_player_X2','away_player_X3','away_player_X4','away_player_X5',
                                      'away_player_X6','away_player_X7','away_player_X8','away_player_X9','away_player_X10','away_player_X11',
                                     'home_player_Y1','home_player_Y2','home_player_Y3','home_player_Y4','home_player_Y5','home_player_Y6',
                                     'home_player_Y7','home_player_Y8','home_player_Y9','home_player_Y10','home_player_Y11',
                                     'away_player_Y1','away_player_Y2','away_player_Y3','away_player_Y4','away_player_Y5','away_player_Y6',
                                     'away_player_Y7','away_player_Y8','away_player_Y9','away_player_Y10','away_player_Y11'])
df = pd.merge(df, df_matches, left_on = 'match_id', right_on = 'id')


In [105]:
#exclude games with red cards #
df_cards = pd.read_csv('data/matches_with_red_card.csv')
print('before red:', len(df))
df = pd.merge(df,df_cards,how='outer', left_on = 'id', right_on = 'match_id', indicator = True)
df = df[df['_merge'] == 'left_only']
print('after red:', len(df))

df = df.drop(columns=['card_type', '_merge', 'Unnamed: 0'])

df['is_home_team'] = df.team_id == df.home_team_api_id

df.to_csv('check.csv')


before red: 3780
after red: 2924


# Create team ratings dataset

In [108]:
# construct a dictionary of players' ratings for different timestamps

d = {}
df_players = pd.read_sql("""SELECT * FROM player_attributes;""", conn)
df_players['date'] = df_players['date'].astype('datetime64[ns]') 
ips = df_players.player_api_id.unique()
for ip in ips:
    df = df_players[df_players['player_api_id'] == ip]
    d[ip] = dict(zip(df.date,df.overall_rating))
#df_players
#date = df_players['date'].astype('date')
#d = dict(zip(df_players.player_api_id,df_players.overall_rating))
#date
d

{505942: {Timestamp('2016-02-18 00:00:00'): 67.0,
  Timestamp('2015-11-19 00:00:00'): 67.0,
  Timestamp('2015-09-21 00:00:00'): 62.0,
  Timestamp('2015-03-20 00:00:00'): 61.0,
  Timestamp('2007-02-22 00:00:00'): 61.0},
 155782: {Timestamp('2016-04-21 00:00:00'): 74.0,
  Timestamp('2016-04-07 00:00:00'): 74.0,
  Timestamp('2016-01-07 00:00:00'): 73.0,
  Timestamp('2015-12-24 00:00:00'): 73.0,
  Timestamp('2015-12-17 00:00:00'): 73.0,
  Timestamp('2015-10-16 00:00:00'): 73.0,
  Timestamp('2015-09-25 00:00:00'): 74.0,
  Timestamp('2015-09-21 00:00:00'): 73.0,
  Timestamp('2015-01-09 00:00:00'): 71.0,
  Timestamp('2014-12-05 00:00:00'): 71.0,
  Timestamp('2014-11-07 00:00:00'): 71.0,
  Timestamp('2014-09-18 00:00:00'): 70.0,
  Timestamp('2014-05-02 00:00:00'): 70.0,
  Timestamp('2014-04-04 00:00:00'): 70.0,
  Timestamp('2014-03-14 00:00:00'): 70.0,
  Timestamp('2013-12-13 00:00:00'): 70.0,
  Timestamp('2013-11-08 00:00:00'): 70.0,
  Timestamp('2013-10-04 00:00:00'): 69.0,
  Timestamp('2013

In [109]:
# construct a dictionary of team ratings

d_power = {}
df_check = pd.read_csv('data/check.csv')
seasons = df_check.season.unique()
#print(seasons[0])
teams = set().union(*[df_check.home_team_api_id.unique(),df_check.away_team_api_id.unique()])
print(len(teams))
#print(teams[0])
for team in list(teams):
    d_power[team] = {}
    #print(team)
    for s in seasons:
        d_power[team][s] = {'field':0, 'gk':0}
        power = 0
        num_players = 0
        gk_power = 0
        num_gk = 0
        year_1, year_2 = s.split('/')
        #print(team,s)
        df = df_check[(df_check.home_team_api_id == team) | (df_check.away_team_api_id == team)]
        df = df[df['season'] == s]
        for i, row in df.iterrows():
            home_or_away = 'home' if row.home_team_api_id == team else 'away'
            for c in [f'{home_or_away}_player_{i}' for i in range(1,12)]:
                #print(c)
                if pd.notna(row[c]):
                    dates = list(d[row[c]].keys())
                    time_to_compare = pd.Timestamp(f'{year_2}-01-01T12')
                    chosen_date = dates[np.argmin([abs(date - time_to_compare) for date in dates])]
                    #print([abs(date - time_to_compare) for date in dates])
                    #print(chosen_date)
                    if  c == f'{home_or_away}_player_1':
                        gk_power += d[row[c]][chosen_date]
                        num_gk +=1
                    else:
                        power += d[row[c]][chosen_date]
                        num_players+=1
        if num_players != 0: 
            d_power[team][s]['field'] = power/num_players
        else:
            d_power[team][s]['field'] = np.nan
        if num_gk != 0: 
            d_power[team][s]['gk'] = gk_power/num_gk
        else:
            d_power[team][s]['gk'] = np.nan
                    
                    



164


In [111]:

#column_names = ["team_id", "2008/2009_overall", "2009/2010_overall","2010/2011_overall","2011/2012_overall","2012/2013_overall","2013/2014_overall","2014/2015_overall","2015/2016_overall",
#                "2008/2009_gk", "2009/2010_gk", "2010/2011_gk", "2011/2012_gk", "2012/2013_gk", "2013/2014_gk", "2014/2015_gk", "2015/2016_gk"]

column_names = ["team_id", "season", "field players rating", "gk rating"]
df_power_ratings = pd.DataFrame(columns = column_names)
i = 0 
for team in list(d_power.keys()):
    for season in list(d_power[team].keys()):
        df_power_ratings.loc[i] = [team] + [season] + [d_power[team][season]['field']] + [d_power[team][season]['gk']]
        i +=1

#df_power_ratings[df_power_ratings['2015/2016_gk'] > 87]

df_power_ratings.to_csv('teams_ratings.csv')

# Create team attributes dataset

In [112]:
# construct a dictionary of team attributes for different timestamps

d = {}
# df_teams = pd.read_csv("team_attributes.csv")
df_teams = pd.read_sql("""SELECT * FROM Team_Attributes;""", conn)
columns = list(df_teams.columns)
df_teams['date'] = df_teams['date'].astype('datetime64[ns]') 
ips = df_teams.team_api_id.unique()
for ip in ips:
    d[ip] = {}
    df = df_teams[df_teams['team_api_id'] == ip]
    for i,row in df.iterrows():
        d[ip][row['date']] = {}
        for c in columns:
            if c not in ['id','team_fifa_api_id', 'team_api_id', 'date']:
                d[ip][row['date']][c] = row[c]
#df_players
#date = df_players['date'].astype('date')
#d = dict(zip(df_players.player_api_id,df_players.overall_rating))
#date
d

{9930: {Timestamp('2010-02-22 00:00:00'): {'buildUpPlaySpeed': 60,
   'buildUpPlaySpeedClass': 'Balanced',
   'buildUpPlayDribbling': nan,
   'buildUpPlayDribblingClass': 'Little',
   'buildUpPlayPassing': 50,
   'buildUpPlayPassingClass': 'Mixed',
   'buildUpPlayPositioningClass': 'Organised',
   'chanceCreationPassing': 60,
   'chanceCreationPassingClass': 'Normal',
   'chanceCreationCrossing': 65,
   'chanceCreationCrossingClass': 'Normal',
   'chanceCreationShooting': 55,
   'chanceCreationShootingClass': 'Normal',
   'chanceCreationPositioningClass': 'Organised',
   'defencePressure': 50,
   'defencePressureClass': 'Medium',
   'defenceAggression': 55,
   'defenceAggressionClass': 'Press',
   'defenceTeamWidth': 45,
   'defenceTeamWidthClass': 'Normal',
   'defenceDefenderLineClass': 'Cover'},
  Timestamp('2014-09-19 00:00:00'): {'buildUpPlaySpeed': 52,
   'buildUpPlaySpeedClass': 'Balanced',
   'buildUpPlayDribbling': 48.0,
   'buildUpPlayDribblingClass': 'Normal',
   'buildUpPla

In [113]:
# construct a dictionary of team ratings

d_attributes = {}
df_check = pd.read_csv('data/check.csv')
seasons = df_check.season.unique()
#print(seasons[0])
teams = set().union(*[df_check.home_team_api_id.unique(),df_check.away_team_api_id.unique()])
print(len(teams))
#print(teams[0])
for team in list(teams):
    d_attributes[team] = {}
    #print(team)
    for s in seasons:
        d_power[team][s] = {}
        year_1, year_2 = s.split('/')
        time_to_compare = pd.Timestamp(f'{year_2}-01-01T12')
        dates = list(d[team].keys())
        chosen_date = dates[np.argmin([abs(date - time_to_compare) for date in dates])]
        d_attributes[team][s] = d[team][chosen_date]
                    

164


In [115]:

column_names = [
 'team_id',
 'season',
 'buildUpPlaySpeed',
 'buildUpPlaySpeedClass',
 'buildUpPlayDribbling',
 'buildUpPlayDribblingClass',
 'buildUpPlayPassing',
 'buildUpPlayPassingClass',
 'buildUpPlayPositioningClass',
 'chanceCreationPassing',
 'chanceCreationPassingClass',
 'chanceCreationCrossing',
 'chanceCreationCrossingClass',
 'chanceCreationShooting',
 'chanceCreationShootingClass',
 'chanceCreationPositioningClass',
 'defencePressure',
 'defencePressureClass',
 'defenceAggression',
 'defenceAggressionClass',
 'defenceTeamWidth',
 'defenceTeamWidthClass',
 'defenceDefenderLineClass']
df_team_attributes = pd.DataFrame(columns = column_names)
i = 0 
for team in list(d_attributes.keys()):
    for season in list(d_attributes[team].keys()):
        df_team_attributes.loc[i] = [team] + [season] + list(d_attributes[team][season].values())
        i +=1


#df_team_attributes.to_csv('teams_details.csv')


# Merge team ratings and attributes datasets and create one dataset from them

In [116]:
df_merge = pd.merge(df_team_attributes,df_power_ratings, right_on = ['team_id','season'], left_on = ['team_id','season'])


df_merge = df_merge.dropna(axis='rows', subset = ['gk rating'])

df_team_name = pd.read_sql("""SELECT * FROM team;""", conn)
# df_team_name = pd.read_csv('team.csv')
df_team_name = df_team_name.drop(columns=['team_short_name', 'team_fifa_api_id', 'id'])
df_merge = pd.merge(df_team_name,df_merge, right_on = ['team_id'], left_on = ['team_api_id'])
df_merge = df_merge.drop(columns = ['team_api_id', 'buildUpPlayDribbling'])
df_merge.to_csv('data/team_full_details.csv')
#df_merge.isna().sum()

# Merge the team ratings and attributes dataset with the mathces dataset 
# This is the main dataset for our analysis

In [117]:
# add df_merge to the relevant matches dataset:
new_columns = ['buildUpPlaySpeed', 'buildUpPlaySpeedClass', 'buildUpPlayDribblingClass', 
               'buildUpPlayPassing', 'buildUpPlayPassingClass', 'buildUpPlayPositioningClass', 
               'chanceCreationPassing', 'chanceCreationPassingClass', 'chanceCreationCrossing', 
               'chanceCreationCrossingClass', 'chanceCreationShooting', 'chanceCreationShootingClass', 
               'chanceCreationPositioningClass', 'defencePressure', 'defencePressureClass', 
               'defenceAggression', 'defenceAggressionClass', 'defenceTeamWidth', 'defenceTeamWidthClass', 
               'defenceDefenderLineClass', 'field players rating', 'gk rating']

df_check = pd.read_csv("data/check.csv")
df_teams_matches = pd.merge(df_check, df_merge, right_on=['team_id','season'], left_on=['home_team_api_id','season'])

for c in new_columns:
    df_teams_matches['curr_{}'.format(c)] = ""
    df_teams_matches['rival_{}'.format(c)] = ""

    df_teams_matches['dominating_{}'.format(c)] = ""
    df_teams_matches['dominated_{}'.format(c)] = ""

# print(df_teams_matches.columns)

for i, row in df_teams_matches.iterrows():
    if row['T'] == True:
        # T  -  not T
        if row['team_id_x'] == row['team_id_y']:
            for c in new_columns:
                df_teams_matches.loc[i, 'dominating_{}'.format(c)] = row[c]
        else:
            for c in new_columns:
                df_teams_matches.loc[i, 'dominated_{}'.format(c)] = row[c]
    else:
        if row['team_id_x'] == row['team_id_y']:
            for c in new_columns:
                df_teams_matches.loc[i, 'dominated_{}'.format(c)] = row[c]
        else:
            for c in new_columns:
                df_teams_matches.loc[i, 'dominating_{}'.format(c)] = row[c]
        
df_teams_matches = df_teams_matches.drop(columns = new_columns)


df_teams_matches = pd.merge(df_teams_matches,df_merge,right_on = ['team_id','season'], left_on = ['away_team_api_id','season'])

for i, row in df_teams_matches.iterrows():
    if row['T'] == True:
        if row['team_id_x'] == row['team_id']:
            for c in new_columns:
                df_teams_matches.loc[i, 'dominating_{}'.format(c)] = row[c]
        else:
            for c in new_columns:
                df_teams_matches.loc[i, 'dominated_{}'.format(c)] = row[c]
    else:
        if row['team_id_x'] == row['team_id']:
            for c in new_columns:
                df_teams_matches.loc[i, 'dominated_{}'.format(c)] = row[c]
        else:
            for c in new_columns:
                df_teams_matches.loc[i, 'dominating_{}'.format(c)] = row[c]
        
df_teams_matches = df_teams_matches.drop(columns = new_columns)

####TODO: remove useless columns and rename some columns before saving dataframe ####

df_teams_matches.to_csv("data/Mathces_with_full_team_details.csv")

In [118]:
try:
    df_teams_matches = df_teams_matches.drop('team_id', axis='columns', errors='ignore')
    df_teams_matches = df_teams_matches.rename({'match_id_x': 'match_id', 'team_id_x': 'team_id', 'team_long_name': 'team_name'}, axis='columns')
except Exception as e:
    print('exception7', e)

try:
    bets_cols = ['B365', 'BW', 'IW', 'LB', 'WH', 'SJ', 'VC', 'GB', 'BS']

    df_teams_matches['draw_bets'] = df_teams_matches[[b+'D' for b in bets_cols]].mean(axis='columns')

    home_mask = df_teams_matches['is_home_team']
    df_teams_matches.loc[home_mask, 'curr_bets'] = df_teams_matches.loc[home_mask, [b+'H' for b in bets_cols]].mean(axis='columns')
    df_teams_matches.loc[home_mask, 'rival_bets'] = df_teams_matches.loc[home_mask, [b+'A' for b in bets_cols]].mean(axis='columns')
    df_teams_matches.loc[~home_mask, 'curr_bets'] = df_teams_matches.loc[~home_mask, [b+'A' for b in bets_cols]].mean(axis='columns')
    df_teams_matches.loc[~home_mask, 'rival_bets'] = df_teams_matches.loc[~home_mask, [b+'H' for b in bets_cols]].mean(axis='columns')

except Exception as e:
    print('exception1', e)
try:
    for c in ['_team_api_id']:
        df_teams_matches.loc[home_mask, 'curr'+c] = df_teams_matches['home'+c]
        df_teams_matches.loc[home_mask, 'rival'+c] = df_teams_matches['away'+c]
        df_teams_matches.loc[~home_mask, 'curr'+c] = df_teams_matches['away'+c]
        df_teams_matches.loc[~home_mask, 'rival'+c] = df_teams_matches['home'+c]

except Exception as e:
    print('exception2', e)
    
try:
    dominating_mask = df_teams_matches['T']
    for c in new_columns:
        df_teams_matches.loc[dominating_mask, 'curr_'+c] = df_teams_matches['dominating_'+c]
        df_teams_matches.loc[dominating_mask, 'rival_'+c] = df_teams_matches['dominated_'+c]
        df_teams_matches.loc[~dominating_mask, 'curr_'+c] = df_teams_matches['dominated_'+c]
        df_teams_matches.loc[~dominating_mask, 'rival_'+c] = df_teams_matches['dominating_'+c]
    
except Exception as e:
    print('exception3', e)
    
try:
    df_teams_matches['curr_shots_1'] = df_teams_matches['shots_on_first_half']
    df_teams_matches['rival_shots_1'] = df_teams_matches.apply(lambda r: df_teams_matches[(df_teams_matches['match_id']==r['match_id']) & 
                                                                                          (df_teams_matches['curr_team_api_id']!=r['curr_team_api_id'])]
                                                           ['curr_shots_1'].iloc[0], axis='columns')
    
except Exception as e:
    print('exception4', e)


# try:
df_teams_matches['date'] = pd.to_datetime(df_teams_matches['date'])
df_teams_matches['month'] = df_teams_matches['date'].dt.month

teams = pd.read_sql("""SELECT * FROM Team;""", conn)
df_teams_matches = df_teams_matches.merge(teams[['team_api_id', 'team_long_name']], left_on='team_id', right_on='team_api_id').drop('team_api_id', axis='columns', errors='ignore')

country = pd.read_sql("""SELECT * FROM Country;""", conn)
df_teams_matches = df_teams_matches.merge(country.rename({'name': 'country_name'}, axis='columns'), left_on='country_id', right_on='id').drop('id', axis='columns', errors='ignore')

card = pd.read_csv('data/card_detail.csv')
match_cards = card[card['elapsed']<=45].groupby('match_id', as_index=False).size().rename({'size': 'cards'}, axis='columns')
print(df_teams_matches.columns, match_cards.columns)
df_teams_matches = df_teams_matches.merge(match_cards, on='match_id')

foul = pd.read_csv('data/foulcommit_detail.csv')
match_fouls = foul.groupby('match_id', as_index=False).size().rename({'size': 'fouls'}, axis='columns')
df_teams_matches = df_teams_matches.merge(match_fouls, on='match_id')    



remove_cols = ['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 
               'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11', 
               'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5', 
               'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11', 
               'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 
               'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 
               'dominating_buildUpPlaySpeed', 'dominated_buildUpPlaySpeed', 'dominating_buildUpPlaySpeedClass', 'dominated_buildUpPlaySpeedClass', 
               'dominating_buildUpPlayDribblingClass', 'dominated_buildUpPlayDribblingClass', 'dominating_buildUpPlayPassing', 'dominated_buildUpPlayPassing', 
               'dominating_buildUpPlayPassingClass', 'dominated_buildUpPlayPassingClass', 'dominating_buildUpPlayPositioningClass', 'dominated_buildUpPlayPositioningClass', 
               'dominating_chanceCreationPassing', 'dominated_chanceCreationPassing', 'dominating_chanceCreationPassingClass', 'dominated_chanceCreationPassingClass', 
               'dominating_chanceCreationCrossing', 'dominated_chanceCreationCrossing', 'dominating_chanceCreationCrossingClass', 'dominated_chanceCreationCrossingClass', 
               'dominating_chanceCreationShooting', 'dominated_chanceCreationShooting', 'dominating_chanceCreationShootingClass', 'dominated_chanceCreationShootingClass', 
               'dominating_chanceCreationPositioningClass', 'dominated_chanceCreationPositioningClass', 'dominating_defencePressure', 'dominated_defencePressure', 
               'dominating_defencePressureClass', 'dominated_defencePressureClass', 'dominating_defenceAggression', 'dominated_defenceAggression', 
               'dominating_defenceAggressionClass', 'dominated_defenceAggressionClass', 'dominating_defenceTeamWidth', 'dominated_defenceTeamWidth', 
               'dominating_defenceTeamWidthClass', 'dominated_defenceTeamWidthClass', 'dominating_defenceDefenderLineClass', 'dominated_defenceDefenderLineClass', 
               'dominating_field players rating', 'dominated_field players rating', 'dominating_gk rating', 'dominated_gk rating',
               'Unnamed: 0', 'index','away_team_api_id', 'home_team_api_id', 'id', 'league_id', 'match_id_y', 'team_id', 'team_id_y', 'team_long_name_x', 'team_long_name_y']

try:
    df_teams_matches_save = df_teams_matches.drop(remove_cols, axis='columns', errors='ignore')
    df_teams_matches_save.to_csv(f"data/Mathces_with_full_team_details_{threshold_over_50}.csv")
    pass
except Exception as e:
    print('exception6', e)

print('done')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (7,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Unnamed: 0', 'index', 'match_id', 'team_id', 'shots_on_first_half',
       'shots_on_second_half', 'goals_first_half', 'T', 'id_x', 'country_id',
       ...
       'curr_bets', 'rival_bets', 'curr_team_api_id', 'rival_team_api_id',
       'curr_shots_1', 'rival_shots_1', 'month', 'team_long_name', 'id_y',
       'country_name'],
      dtype='object', length=169) Index(['match_id', 'cards'], dtype='object')


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (0,12,17,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


done
